# SQL MINI PROJECT

### Imports

In [1]:
from sqlalchemy import create_engine
import pandas as pd

### Connect to SQLite DB and Create engine

In [2]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


### Look over Imports of Datatables

In [4]:
df_raw_Bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
df_raw_Facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)
df_raw_Members = pd.read_sql_query("SELECT * FROM Members", engine)

#### Bookings Table

In [5]:
print(df_raw_Bookings.head())
df_raw_Bookings.shape

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


(4043, 5)

##### Check Number of 'Guests' in Bookings Table

In [6]:
df_raw_Bookings[df_raw_Bookings.memid ==  0].count()

bookid       883
facid        883
memid        883
starttime    883
slots        883
dtype: int64

#### Facilities Table

In [7]:
print(df_raw_Facilities.head())
df_raw_Facilities.shape

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


(9, 6)

#### Members Table

In [8]:
print(df_raw_Members.head())
df_raw_Members.shape

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


(31, 8)

### Question 10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! */

In [9]:
sql_query10 = '''
SELECT name AS Facility_Name,
    SUM(CASE WHEN b.memid = 0 THEN f.guestcost
        ELSE f.membercost END) AS Revenue
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
GROUP BY name
ORDER BY Revenue DESC'''
df_q10 = pd.read_sql_query(sql_query10, engine,index_col = 'Facility_Name')
print(df_q10)

                 Revenue
Facility_Name           
Massage Room 1   20807.9
Massage Room 2    6987.3
Squash Court      4970.0
Tennis Court 2    4205.0
Tennis Court 1    4040.0
Badminton Court    604.5
Pool Table         265.0
Snooker Table      115.0
Table Tennis        90.0


### Question 11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [10]:
sql_query11 = '''SELECT (m1.surname || ',' || m1.firstname) as Member_name, (m2.surname || ',' || m2.firstname) as RecommendedBy_name
FROM Members as m1
LEFT JOIN Members as m2
ON m1.recommendedby = m2.memid
ORDER BY RecommendedBy_name
'''
df_q11 = pd.read_sql_query(sql_query11, engine,index_col = 'Member_name')
print(df_q11)

                        RecommendedBy_name
Member_name                               
GUEST,GUEST                           None
Smith,Darren                          None
Smith,Tracy                           None
Rownam,Tim                            None
Tracy,Burton                          None
Farrell,Jemima                        None
Farrell,David                         None
Tupperware,Hyacinth                   None
Smith,Darren                          None
Sarwin,Ramnaresh            Bader,Florence
Coplin,Joan                  Baker,Timothy
Genting,Matthew             Butters,Gerald
Baker,Timothy               Farrell,Jemima
Pinker,David                Farrell,Jemima
Rumney,Henrietta           Genting,Matthew
Jones,Douglas                  Jones,David
Dare,Nancy                 Joplette,Janice
Jones,David                Joplette,Janice
Hunt,John                Purview,Millicent
Boothe,Tim                      Rownam,Tim
Joplette,Janice               Smith,Darren
Butters,Ger

### Question 12: Q12: Find the facilities with their usage by member, but not guests

In [11]:
sql_query12 = '''
SELECT b.memid, f.name as Facilitiy_name,
    COUNT(b.bookid) AS Usage      
FROM Bookings as b
INNER JOIN Facilities as f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY b.memid, f.name
ORDER BY b.memid'''
df_q12 = pd.read_sql_query(sql_query12, engine, index_col = 'memid')
print(df_q12)

        Facilitiy_name  Usage
memid                        
1      Badminton Court    132
1       Massage Room 1     28
1           Pool Table     28
1        Snooker Table     12
1         Squash Court     14
...                ...    ...
35      Tennis Court 2      4
36     Badminton Court      2
36      Massage Room 1      2
36        Table Tennis      2
36      Tennis Court 1      1

[202 rows x 2 columns]


In [12]:
#Check that Guests were not included
sum(df_q12.Usage)

3160

### Q13: Find the facilities usage by month, but not guests

In [13]:
sql_query13 = '''
SELECT strftime('%m', starttime) AS Month, 
    f.name as Facilitiy_name,
    COUNT(b.bookid) AS Usage      
FROM Bookings as b
INNER JOIN Facilities as f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY Month, f.name
ORDER BY Month'''
df_q13 = pd.read_sql_query(sql_query13, engine, index_col = 'Facilitiy_name')
print(df_q13)

                Month  Usage
Facilitiy_name              
Badminton Court    07     51
Massage Room 1     07     77
Massage Room 2     07      4
Pool Table         07    103
Snooker Table      07     68
Squash Court       07     23
Table Tennis       07     48
Tennis Court 1     07     65
Tennis Court 2     07     41
Badminton Court    08    132
Massage Room 1     08    153
Massage Room 2     08      9
Pool Table         08    272
Snooker Table      08    154
Squash Court       08     85
Table Tennis       08    143
Tennis Court 1     08    111
Tennis Court 2     08    109
Badminton Court    09    161
Massage Room 1     09    191
Massage Room 2     09     14
Pool Table         09    408
Snooker Table      09    199
Squash Court       09     87
Table Tennis       09    194
Tennis Court 1     09    132
Tennis Court 2     09    126


In [14]:
#Check that Guests were not included
sum(df_q13.Usage)

3160